In [1]:
!pip install wikipedia-api
!pip install pyvis
!pip install eventlet

In [2]:
import wikipediaapi
import pandas as pd
import numpy as np

In [3]:
# On ne considère que le Wikipedia en français

wiki_wiki = wikipediaapi.Wikipedia("fr", extract_format=wikipediaapi.ExtractFormat.WIKI)

In [4]:
# On va commencer par écrire une fonction qui retourne la liste des voisins d'une page donnée

def get_neighbors(name):
    
    page = wiki_wiki.page(name)
    return list(page.links.keys())

In [5]:
# On doit ensuite trouver les voisins des voisins, les ajouter dans une pile de pages à parcourir, puis trouver leurs voisins et recommencer, 
# et ainsi de suite jusqu'à ce que la pile soit vide (l'élément d'indice 0 est le fond de la pile)
# On parle alors de parcours en profondeur du graphe (qu'on peut voir comme un arbre ici)
# On va représenter le graphe sous forme d'un dictionnaire d'adjacence, de la forme {"sommet" : [voisins]}

# Première version pour évaluer le type des liens présents dans des pages quelconques :

def get_tree(start):
    i = 0
    pile = [start]
    tree = {}
    while pile != [] and i < 50:
        node = pile.pop()
        i += 1
        print(node) # On va visualiser les noms de pages qu'on traite
        tree[node] = []
        for neighbor in get_neighbors(node):
            tree[node].append(neighbor)
            if neighbor not in tree.keys():
                pile.append(neighbor)
    return tree

In [ ]:
# On fait un appel à la fonction get_tree pour regarder ce qu'il se passe

get_tree("Python_(langage)")

In [6]:
# Problème : On voit qu'il y a beaucoup de liens qui mènent vers des pages "indésirables", comme les modèles, les discussions, les aides etc

# On va donc réaliser un filtrage d'expression régulière sur le nom des pages, à l'aide du module re :

import re

def filtering(name):
    if re.match(r"Portail:.*", name):
        return False
    if re.match(r"Discussion Projet:.*", name):
        return False
    if re.match(r"Discussion:.*", name):
        return False
    if re.match(r"Module:.*", name):
        return False
    if re.match(r"Projet:.*", name):
        return False
    if re.match(r"Modèle:.*", name):
        return False
    if re.match(r"Aide:.*", name):
        return False
    if re.match(r"Catégorie:.*", name):
        return False
    if re.match(r"Utilisateur:.*", name):
        return False
    if re.match(r"Utilisatrice:.*", name):
        return False
    if re.match(r"Discussion Utilisateur:.*", name):
        return False
    if re.match(r"Sujet:.*", name):
        return False
    if re.match(r"Référence:.*", name):
        return False
    if re.match(r"Discussion Wikipédia:.*", name):
        return False
    if re.match(r"Fichier:.*", name):
        return False
    if re.match(r"Discussion modèle:.*", name):
        return False
    if re.match(r"Discussion Portail:.*", name):
        return False
    if re.match(r"Discussion catégorie:.*", name):
        return False
    if re.match(r"Wikipédia:.*", name):
        return False
    if re.match(r"Discussion utilisateur:.*", name):
        return False
    return True

In [9]:
# On rajoute une ligne dans la fonction :

import numpy as np # On a besoin des fonction numpy sur les tableaux

def get_10000_nodes(pile, tree):
    i = 0
    while pile != [] and i<10000:
        global node
        node = pile.pop()
        if node not in tree.keys():
            # print(node)
            tree[node] = [[]]
            i += 1
            neighbors = get_neighbors(node)
            if len(neighbors)>0:
                for neighbor in neighbors:
                    if filtering(neighbor) and len(neighbor)>1:
                        tree[node].append(neighbor)
                        if neighbor not in tree.keys():
                            pile.append(neighbor)
    return list(np.unique(np.array(pile))), tree
import requests
import eventlet # Pour réduire la possibilité de timeout à la lecture

global pile, tree # Pour pouvoir garder la progression en cas d'erreur, seule la page d'erreur est perdue, pas très grave

def get_tree_clean(start = "Python_(langage)", pile = [], tree = {}): #On cherche tout le réseau de pages, en partant de la page de Python
    j = 0 # Simple compteur pour savoir la vitesse approximative en exécution
    with eventlet.Timeout(40000): #Pour empêcher les timeouts
        while pile != []: # On traite tous les sommets possibles
            pile, tree = get_10000_nodes(pile, tree) # On actualise la valeur toutes les 10 000 pages 
            j += 1
            print(j)

    return tree

In [95]:
# Pour visualiser le réseau des pages Wikipédia, on va devoir transformer le dictionnaire d'adjacence en dataframe pandas
# Le dataframe comporte 4 colonnes : "Source", "Target", "Type" et "Weight"
# Le graphe étant orienté, le type d'arête est "Directed"

def create_dataframe(tree, nb_nodes):
    data = {"Source" : [], "Target" : [], "Size" : [], "Weight" : [], "Same_Category" = []}
    i = 0
    for node in tree.keys():
        i+=1
        if i == nb_nodes:
            return pd.DataFrame(data)
        else:
            for neighbor in tree[node]:
                if neighbor in tree.keys():
                    weight = len(tree[neighbor])
                else:
                    weight = 0
                data["Source"].append(node)
                data["Target"].append(neighbor)
                data["Size"].append(len(tree[node]))
                data["Weight"].append(weight)

In [173]:
# On visualise alors le graphe, en utilisant le module PyVis :

from pyvis.network import Network

def visualize(data):
    net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white", notebook = True)
    net.barnes_hut()
    sources = data["Source"]
    targets = data["Target"]
    weights = data["Weight"]
    sizes = data["Size"]

    edge_data = zip(sources, targets, weights, sizes)

    for e in edge_data:
        src = e[0]
        dst = e[1]
        w = e[2]
        sz = e[3]

        net.add_node(src, src, size = sz/10, title = src)
        net.add_node(dst, dst, size = w/10, stitle = dst)
        net.add_edge(src, dst, value = w**2)
    net.show("Wiki_map.html")

In [3]:
tree = pd.read_pickle("https://minio.lab.sspcloud.fr/cheitz/tree.pickle")

In [147]:
data = create_dataframe(tree, 100000)

In [174]:
data2 = data[:100000]

In [175]:
visualize(data2)

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [71]:
def initialize(tree, deb):
    d = {}
    sommets = list(tree.keys())
    for v in tree.values():
        sommets += v
    sommets = list(np.unique(sommets))
    for s in sommets:
        d[s] = (np.inf, None)
    d[deb] = (0, None)
    return d

In [5]:
def find_min(d, Q):
    mini = np.inf
    sommet = None
    for s in Q.keys():
        if d[s][0]<mini:
            mini = d[s][0]
            sommet = s
    return sommet

In [58]:
def maj_distances(d, s1, s2):
    if d[s2][0]>d[s1][0]+1:
        d[s2] = (d[s1][0]+1,s1)

In [67]:
def dijkstra(tree, deb, fin):
    d = initialize(tree, deb)
    Q = tree.copy()
    s1 = None
    while Q!={} and s1!=fin:
        s1 = find_min(d, Q)
        if s1 == None:
            return None
        Q.pop(s1)
        for s2 in tree[s1]:
            if s2 in d.keys():
                maj_distances(d, s1, s2)
    return d

In [144]:
def chemin(tree, deb, fin):
    d = dijkstra(tree, deb, fin)
    if d == None:
        print("Pas de chemin trouvé entre vos 2 sommets :( ")
        pass
    else:
        A = []
        s = fin
        while s!=deb:
            A = [s]+A
            s = d[s][1]
        return [deb]+A

In [134]:
nodes = list(tree.keys())
len(nodes)

605470

In [140]:
nodes = list(np.random.choice(nodes, size = 10000, replace = False))
nodes.append("Python_(langage)")
nodes.append("États-Unis")
tree_100k = {k : tree[k] for k in nodes}

In [136]:
deb, fin = np.random.choice(nodes), np.random.choice(nodes)

In [141]:
deb = "Python_(langage)"

In [142]:
fin = "États-Unis"

In [145]:
chemin(tree_100k, deb, fin)

['Python_(langage)', 'États-Unis']

In [ ]:

#tentative d'implémentation de Dijkstra

#Créer la matrice adjacente des distances entre les pages

def Matrice_Adj(node):
    tree = get_tree(node)
    Mat_Adj = np.zeros(len(tree),len(tree))
    for i in range(1,len(tree[node])+1):
        Mat_Adj[0,i]=1
    #intégrer ligne par ligne les 1 indiquant un lien (donc faire deux boucles for l'une dans l'autre)

def Dijkstra(pt_depart, pt_arrivee):
    Mat_Adj_Dji = Matrice_Adj(pt_depart)
    #implémenter l'algo de Djikstra
    

In [ ]:
# Création de l'interface graphique
from tkinter import *
import tkinter.font as font

#Création de la fenêtre et de son titre
fenêtre=Tk()
fenêtre.title("Plus court chemin Wikipedia")
fenêtre.config(bg = "blue") #couleur de l'arrière-plan
fenêtre.geometry("800x600") #dimensions de la fenêtre


#Création d'un label pour la saisie des sommets à relier
SommetLabel = Label(fenêtre, text="Saisir les pages wikipedia de départ et d'arrivée : ", bg = "blue", fg='white')
SommetLabel['font'] = font.Font(family='Times New Roman', size=18, weight="bold")
SommetLabel.pack(pady=10)
    
    
#Création des zones de saisie
sommet1 = StringVar()
sommet2 = StringVar()

sommet1.set("page de départ, ex : Python(langage)")
sommet2.set("page d'arrivée, ex : Chocolat")

saisieSommet1 = Entry(fenêtre, textvariable=sommet1, width=35)
saisieSommet2 = Entry(fenêtre, textvariable=sommet2, width=35)

saisieSommet1['font'] = font.Font(family='Times New Roman', size=15)
saisieSommet2['font'] = font.Font(family='Times New Roman', size=15)

saisieSommet1.pack(pady=10)
saisieSommet2.pack(pady=5)

#Récupération des entrées
départ = saisieSommet1.get()
arrivée = saisieSommet2.get()


#Création d'un bouton
bouton1 = Button(fenêtre, text="CALCULER CHEMIN", relief=GROOVE, width=17, height=1, bg="yellow", command=chemin(tree, départ, arrivée)) 
bouton1['font'] = font.Font(family='Times New Roman', size=15, weight="bold")
bouton1.pack(pady=20)


#Label pour afficher le résultat
ResultatLabel = Label(fenêtre, text="Le plus court chemin est : ", bg = "blue", fg='white')
ResultatLabel['font'] = font.Font(family='Times New Roman', size=18, weight="bold")
ResultatLabel.pack(pady=100)